<a href="https://colab.research.google.com/github/AbeerProg/RRDS/blob/main/vaderSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!pip install vaderSentiment
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 582.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 50.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import pandas as pd
import spacy
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load SpaCy English model
nlp = spacy.load('en_core_web_sm')

# Load the dataset
df = pd.read_excel('main.xlsx')

# Display the first few rows of the dataset
print(df.head())


                           text
0           Need more munchkins
1                Nice as always
2            Great little place
3      Service was exceptional!
4  Perfect breakfast everytime.


In [4]:
def preprocess_text(text):
    # Process the text with SpaCy
    doc = nlp(text.lower())  # Lowercase the text

    # Retain words, skip punctuation, and apply lemmatization
    processed_tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

    # Join tokens back into a single string
    processed_text = ' '.join(processed_tokens)
    return processed_text

def vader_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score['compound']  # Return the compound score which is a normalized score between -1 and 1


In [5]:
# Apply the preprocessing function to each review in the DataFrame
df['preprocessed_text'] = df['text'].apply(preprocess_text)

# Apply VADER sentiment analysis to the preprocessed text
df['vader_sentiment'] = df['preprocessed_text'].apply(vader_sentiment)

# Define a function to interpret the sentiment scores
def interpret_vader(compound_score):
    if compound_score >= 0.6:
        return "Strong Positive"
    elif compound_score >= 0.2:
        return "Positive"
    elif compound_score > -0.2:
        return "Neutral"
    elif compound_score > -0.6:
        return "Negative"
    else:
        return "Strong Negative"

# Add interpretation column
df['vader_sentiment_category'] = df['vader_sentiment'].apply(interpret_vader)

# Display the DataFrame with the new sentiment columns
print(df[['text', 'preprocessed_text', 'vader_sentiment', 'vader_sentiment_category']])

# Save the updated DataFrame to a new Excel file
df.to_excel('vader_sentiment_output.xlsx', index=False)


                                                    text  \
0                                    Need more munchkins   
1                                         Nice as always   
2                                     Great little place   
3                               Service was exceptional!   
4                           Perfect breakfast everytime.   
...                                                  ...   
25115  Flavors were all pretty good, very fresh clean...   
25116  I usually do not make reviews unless I think i...   
25117  TLDR in terms of Covid-19 safety and general d...   
25118  I've ordered from this restaurant twice, and i...   
25119  I have been working my way through the menu si...   

                                       preprocessed_text  vader_sentiment  \
0                                          need munchkin           0.0000   
1                                                   nice           0.4215   
2                                     great litt